In [1]:
import pickle 
import gensim
import os
import sys
import json
from acrlist import acr
import tensorflow as tf
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.models import load_model

Using TensorFlow backend.


In [3]:
train_data_tweets = []
train_data_acronyms = []
acr_count = 0

for expansion in acr['gg']:
    with open("train_data/"+str(expansion)) as file:
      tweets = file.readlines()
      for tweet in tweets:
        tweet = json.loads(tweet)
        tweet = tweet.lower()
        tweet = tweet.replace(expansion, 'gg')
        train_data_tweets.append(tweet.split())
        train_data_acronyms.append(expansion)

In [4]:
def create_tagged_document(split_tweets):
  for i, tweet in enumerate(split_tweets):
    yield gensim.models.doc2vec.TaggedDocument(tweet, [train_data_acronyms[i]])
    
train_data = list(create_tagged_document(train_data_tweets))
print(train_data[1:2])

[TaggedDocument(words=['@vinijrmadrid', 'yh', 'i', 'agree', 'frenkie', 'is', 'overrated.', 'however,', 'have', 'you', 'ever', 'seen', 'a', 'gg', 'from', 'varane', 'without', 'ramos', 'by', 'his', 'side?'], tags=['good game'])]


In [5]:
model = gensim.models.doc2vec.Doc2Vec(vector_soze=50, min_count=2, epochs=100)
model.build_vocab(train_data)
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

In [6]:
v = model.infer_vector("el classico was gg".split())
res_tup = model.docvecs.most_similar([v])[0]
print("Doc2Vec Vector & Prediction:")
print([v])
print(res_tup)

Doc2Vec Vector & Prediction:
[array([ 3.30729224e-02,  4.25219387e-02,  2.14383215e-01,  2.31502920e-01,
        8.92790109e-02,  2.60980248e-01,  1.90944389e-01, -2.30320748e-02,
        2.91241258e-01,  2.43776634e-01,  1.51321203e-01, -2.26441965e-01,
        5.35409153e-01, -3.85960609e-01,  5.32175064e-01,  8.91531855e-02,
        7.58388713e-02, -1.80071354e-01,  4.46996033e-01, -1.73472285e-01,
        4.09612507e-01,  1.26741797e-01,  6.08486354e-01, -2.49143824e-01,
       -5.57823300e-01, -1.99070483e-01,  5.10574803e-02, -3.76354828e-02,
        3.28518569e-01, -2.47748252e-02, -4.11353230e-01, -6.31013885e-03,
        1.05936393e-01,  2.90370416e-02, -2.32851058e-01,  2.32075751e-01,
       -2.14494064e-01,  2.30996713e-01,  2.46923212e-02, -1.59332559e-01,
       -6.52951375e-02,  4.74129207e-02,  2.25286677e-01, -1.93009842e-02,
        1.13294996e-01,  7.19480872e-01,  2.74504662e-01, -4.52864707e-01,
       -1.41044939e-02,  3.13733280e-01, -3.86825860e-01, -1.67045638e

In [13]:
def classifier(X_train, Y_train, X_test, Y_test):
  #takes Doc2Vec as input layer instead of Word Embeddings, and trains classifiers for each acronym
  tf_model = Sequential()
  tf_model.add(Flatten())
  tf_model.add(Dense(128, activation="relu"))
  tf_model.add(Dense(128, activation="relu"))
  tf_model.add(Dense(64, activation="relu"))
  tf_model.add(Dense(32, activation="relu"))
  tf_model.add(Dense(len(set(Y_train)), activation="softmax"))
  tf_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
  tf_model.fit(X_train, Y_train, batch_size=32, nb_epoch=1, verbose=1)
  score, acc = tf_model.evaluate(X_test, Y_test, verbose=1, batch_size=32)
  print("Score: %.2f" % (score))
  print("Validation Accuracy: %.2f" % (acc))
  return tf_model

In [ ]:
X_train = []
X_test = []
Y_train = []
Y_test = []

for expansion in acr['gg']:
    with open("train_data/"+str(expansion)) as file:
      tweets = file.readlines()
      for tweet in tweets:
        tweet = json.loads(tweet)
        tweet = tweet.lower()
        tweet = tweet.replace(expansion, 'gg')
        X_train.append(list(model.infer_vector(tweet.split())))
        Y_train.append(expansion)
    with open("test_data/"+str(expansion)+".txt") as file:
      tweets = file.readlines()
      for tweet in tweets:
        tweet = json.loads(tweet)
        tweet = tweet.lower()
        tweet = tweet.replace(expansion, 'gg')
        X_test.append(list(model.infer_vector(tweet.split())))
        Y_test.append(expansion)

In [36]:
classifier(X_train, Y_train, X_test, Y_test).save(str('gg')+".h5")

/home/enikhil12/ags/venv/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 52144 arrays: [array([[ 3.97711694e-01],
       [-7.09975839e-01],
       [ 1.76661938e-01],
       [ 6.11979067e-01],
       [-4.60130811e-01],
       [-3.22341956e-02],
       [ 1.20659769e+00],
       [-4.668729...